In [20]:
# Set your username here - use it consistently across all resources
USERNAME = "jiaqi-wikidata"

import datetime
import json

import boto3
import requests

In [21]:
# Try different dates to see how the data changes
DATE_PARAM = "2025-11-26"

date = datetime.datetime.strptime(DATE_PARAM, "%Y-%m-%d")

# Construct the API URL
url = f"https://wikimedia.org/api/rest_v1/metrics/pageviews/top/en.wikipedia.org/all-access/{date.strftime('%Y/%m/%d')}"
print(f"Requesting REST API URL: {url}")

# Make the API request
wiki_server_response = requests.get(url, headers={"User-Agent": "curl/7.68.0"})
wiki_response_status = wiki_server_response.status_code
wiki_response_body = wiki_server_response.text

print(f"Wikipedia REST API Response body: {wiki_response_body[:500]}...")
print(f"Wikipedia REST API Response Code: {wiki_response_status}")

# Validate response
if wiki_response_status != 200:
    raise Exception(f"Received non-OK status code from Wiki Server: {wiki_response_status}")
print(f"Successfully retrieved Wikipedia data, content-length: {len(wiki_response_body)}")

Requesting REST API URL: https://wikimedia.org/api/rest_v1/metrics/pageviews/top/en.wikipedia.org/all-access/2025/11/26
Wikipedia REST API Response body: {"items":[{"project":"en.wikipedia","access":"all-access","year":"2025","month":"11","day":"26","articles":[{"article":"Main_Page","views":6246960,"rank":1},{"article":"Special:Search","views":841768,"rank":2},{"article":"Google_Chrome","views":306870,"rank":3},{"article":"Dancing_with_the_Stars_(American_TV_series)_season_34","views":271802,"rank":4},{"article":"Wikipedia:Featured_pictures","views":260897,"rank":5},{"article":"1989_Tiananmen_Square_protests_and_massacre","views":182695,"rank":6...
Wikipedia REST API Response Code: 200
Successfully retrieved Wikipedia data, content-length: 55942


In [22]:
# Parse the API response and extract top edits
wiki_response_parsed = wiki_server_response.json()
top_views = wiki_response_parsed["items"][0]["articles"]

# Transform to JSON Lines format
current_time = datetime.datetime.now(datetime.timezone.utc)
json_lines = ""
for page in top_views[:5]:
    record = {
        "title": page["article"],
        "views": page["views"],
        "rank": page["rank"],
        "date": date.strftime("%Y-%m-%d"),
        "retrieved_at": current_time.replace(tzinfo=None).isoformat(),
    }
    json_lines += json.dumps(record) + "\n"

print(f"Transformed {len(top_views)} records to JSON Lines")
print(f"First few lines:\n{json_lines[:500]}...")

Transformed 1000 records to JSON Lines
First few lines:
{"title": "Main_Page", "views": 6246960, "rank": 1, "date": "2025-11-26", "retrieved_at": "2025-12-10T16:28:53.095740"}
{"title": "Special:Search", "views": 841768, "rank": 2, "date": "2025-11-26", "retrieved_at": "2025-12-10T16:28:53.095740"}
{"title": "Google_Chrome", "views": 306870, "rank": 3, "date": "2025-11-26", "retrieved_at": "2025-12-10T16:28:53.095740"}
{"title": "Dancing_with_the_Stars_(American_TV_series)_season_34", "views": 271802, "rank": 4, "date": "2025-11-26", "retrieved_at": ...


In [23]:
S3_WIKI_BUCKET = "jiaqi-wikidata"
s3 = boto3.client("s3",region_name="eu-west-1")
bucket_names = [bucket["Name"] for bucket in s3.list_buckets()["Buckets"]]
if S3_WIKI_BUCKET not in bucket_names:
    # LAB 1: Create the bucket if it doesn't exist
    # YOUR SOLUTION COMES HERE =========================
    try:
        response = s3.create_bucket(
            Bucket=S3_WIKI_BUCKET,
            CreateBucketConfiguration={
                'LocationConstraint': 'eu-west-1',
                },
        )
    except Exception as e:
        print(f"❌ Error creating bucket: {str(e)}")
    # ==================================================
    
    print(f"Created new bucket: {S3_WIKI_BUCKET}")
else:
    print(f"Using existing bucket: {S3_WIKI_BUCKET}")

Using existing bucket: jiaqi-wikidata


In [24]:
# Test Lab 1
assert USERNAME != "<username>", "Please set your USERNAME at the top of the notebook"
assert S3_WIKI_BUCKET.endswith("-wikidata"), "Bucket name must end with '-wikidata'"

try:
    s3.head_bucket(Bucket=S3_WIKI_BUCKET)
    print(f"Bucket {S3_WIKI_BUCKET} exists!")
except Exception as e:
    print(f"Bucket {S3_WIKI_BUCKET} not found: {e}")
    raise

Bucket jiaqi-wikidata exists!


In [25]:
# LAB 2: Upload json_lines directly to S3
s3_key = f"raw-views/raw-views-{date.strftime('%Y-%m-%d')}.json"
s3.put_object(
    Bucket=S3_WIKI_BUCKET,
    Key=s3_key,
    Body=json_lines,
)
print(f"Uploaded {len(top_views)} records to s3://{S3_WIKI_BUCKET}/{s3_key}")


Uploaded 1000 records to s3://jiaqi-wikidata/raw-views/raw-views-2025-11-26.json


In [27]:
# Test Lab 2
expected_key = f"raw-views/raw-views-{date.strftime('%Y-%m-%d')}.json"
try:
    s3.head_object(Bucket=S3_WIKI_BUCKET, Key=expected_key)
    print(f"File uploaded successfully to s3://{S3_WIKI_BUCKET}/{expected_key}")
except Exception as e:
    print(f"File not found at s3://{S3_WIKI_BUCKET}/{expected_key}")
    raise

File uploaded successfully to s3://jiaqi-wikidata/raw-views/raw-views-2025-11-26.json
